In [1]:
VERSION

v"1.8.1"

In [2]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 20;

Install Astroshaper package

In [3]:
# using Pkg
# Pkg.add(url="https://github.com/MasanoriKanamaru/Astroshaper")

If you would like to install a new package...

In [4]:
# using Pkg
# Pkg.add("CSV")
# Pkg.update("CSV")
# Pkg.build("CSV")

In [7]:
using DataFrames
using GLMakie
using JLD2

using Revise
using Astroshaper
import SPICE

# SPICE Kernels

In [6]:
"""
Obtain a dataframe of ephemerides 

# Arguments
- `targ`   : Target body name
- `ets`    : AbstractVetor of observer epoch
- `ref`    : Reference frame of output position vector
- `abcorr` : Aberration correction flag
- `obs`    : Observing body name
"""
function spkpos_df(targ, ets::AbstractVector, ref, abcorr, obs)
    df = DataFrame(et=Float64[], x=Float64[], y=Float64[], z=Float64[], lt=Float64[])
    for et in ets
        pos, lt = SPICE.spkpos(targ, et, ref, abcorr, obs)  # pos [km], lt [s]
        pos = SPICE.convrt.(pos, "km", "m")
        push!(df, (et, pos..., lt))
    end
    df
end

function spkpos(targ, ets::AbstractVector, ref, abcorr, obs)
    positions = Vector{Float64}[]
    for et in ets
        pos, lt = SPICE.spkpos(targ, et, ref, abcorr, obs)  # pos [km], lt [s]
        pos = SPICE.convrt.(pos, "km", "m")
        push!(positions, collect(pos))
    end
    positions
end


"""
# Arguments
- `from` : Name of the frame to transform from
- `to`   : Name of the frame to transform to
- `ets`  : Epoch of the rotation matrix

# Return
- `rotate` : A rotation matrix
"""
pxform(from, to, ets) = [SPICE.pxform(from, to, et) for et in ets];

In [61]:
meta_kernel = "/Users/masanorikanamaru/Dropbox/spice/hayabusa2/kernels/mk/hyb2_v03.tm"

SPICE.furnsh(meta_kernel)

In [62]:
et_start = SPICE.utc2et("2018-07-01T00:00:00")
et_end   = SPICE.utc2et("2018-08-01T00:00:00")
step     = 76.3262  # Rotation of 1 deg

et_range = et_start : step : et_end

@show et_range
@show length(et_range);

et_range = 5.836752691841208e8:76.3262:5.863536318683208e8
length(et_range) = 35092


In [64]:
## Indices of et_range to be saved.
## Save only the last rotation.
save_range = findall(et_range .> et_range[end] - 7.63262 * 3600)

@show save_range[begin]
@show save_range[end]
@show length(save_range);

save_range[begin] = 34733
save_range[end] = 35092
length(save_range) = 360


In [65]:
sun_ryugu = spkpos("SUN", et_range, "RYUGU_FIXED", "None", "RYUGU");  # Sun's position in the RYUGU_FIXED frame

In [66]:
RYUGU_TO_J2000 = pxform("RYUGU_FIXED", "J2000", et_range);  # Transformation matrix from RYUGU_FIXED to J2000

In [67]:
SPICE.kclear()

In [68]:
# fig = Figure()
# ax = Axis3(fig[1, 1], aspect=:data)

# scatter!([r[1] for r in sun_ryugu], [r[2] for r in sun_ryugu], [r[3] for r in sun_ryugu], color=:orange, size=2)

# display(fig)

# Shape Model

Note that it will take much time to load a shape model for the first time because each facet must find all facets that are visible from itself for a thermophysical simulation.

In [69]:
#### For the first time
# shapepath = "/Users/masanorikanamaru/Dropbox/Anivid/shape/SHAPE_SFM_49k_v20180804.obj"
# shape = ShapeModel(shapepath; scale=1000, find_visible_facets=true, save_shape=true)

In [70]:
#### After the second time, you can load the saved data (*jld2).
shapepath = "/Users/masanorikanamaru/Dropbox/Anivid/shape/SHAPE_SFM_49k_v20180804.jld2"
shape = ShapeModel(shapepath)

Shape model
-----------
Nodes             : 25350
Faces             : 49152
Surface area      : 2.7410142634530533e6
Volume            : 3.772668304867635e8
Equivalent radius : 448.2496970543
Center-of-Figure  : [0.36087616278246093, 0.16010195831071833, 0.012657636711836645]
Inertia tensor    : 
    | Ixx Ixy Ixz |   [0.0, 0.0, 0.0]
    | Iyx Iyy Iyz | = [0.0, 0.0, 0.0]
    | Izx Izy Izz |   [0.0, 0.0, 0.0]


In [71]:
# draw(shape)

# Thermophsyical Simulation on Ryugu

Thermal peroperties [Okada+2020; Shimaki+2020]

In [72]:
thermo_params = ThermoParams(
    A_B   = 0.04,  # Bolometric Bond albedo
    A_TH  = 0.0,
    k     = 0.1,
    ρ     = 1270.0,
    Cp    = 600.0,
    ϵ     = 1.0,
    t_bgn = et_range[begin],
    t_end = et_range[end],
    Nt    = length(et_range),
    z_max = 0.6,
    Nz    = 41,
    P     = 7.63262 * 3600,
)

Thermophysical parameters
-------------------------
A_B   : 0.04
A_TH  : 0.0
k     : 0.1
ρ     : 1270.0
Cp    : 600.0
ϵ     : 1.0
-------------------------
t_bgn : 5.836752691841208e8
t_bgn : 21241.98757671826 (Normalized by period P)
t_end : 5.863536318683208e8
t_end : 21339.462576718262 (Normalized by period P)
Nt    : 35092
Δt    : 76.3262000000017
Δt    : 0.00277777777777784 (Normalized by period P)
-------------------------
z_max : 0.6
z_max : 2.8186148335177874 (Normalized by skin depth l)
Nz    : 41
Δz    : 0.015
Δz    : 0.07046537083794469 (Normalized by skin depth l)
-------------------------
P     : 27477.432
l     : 0.21287051812296282
Γ     : 276.04347483684523
λ     : 0.0445180519101789
-------------------------


In [74]:
savepath = "/Users/masanorikanamaru/Dropbox/Anivid/TPM_Ryugu.jld2"
run_TPM!(shape, et_range, sun_ryugu, thermo_params, savepath, save_range)

jldopen(savepath, "r+") do file
    file["RYUGU_TO_J2000"] = RYUGU_TO_J2000[save_range]
end;

In [87]:
draw(shape; data=:temperature, colormap=:vik)

GLMakie.Screen(...)

In [78]:
data = load(savepath);

In [171]:
# data["shape"]
# data["et_range"]
# data["sun"]
# data["thermo_params"]
# data["surf_temps"]
# data["forces"]
# data["torques"]

# data["RYUGU_TO_J2000"]

360-element Vector{Vector{Float64}}:
 [-5.445478071426464, -13.407310332541739, 1.5165430089894898]
 [-5.546896974206358, -13.355674887146078, 1.5750789025324705]
 [-5.646604281695755, -13.331027118819776, 1.6030290956922635]
 [-5.781724284054608, -13.318326376616906, 1.612806963871151]
 [-5.949858411386413, -13.263027565674754, 1.6168456541264473]
 [-6.109332041814932, -13.23233824896784, 1.6542182423237137]
 [-6.242204823942307, -13.180186221655683, 1.554026383076671]
 [-6.397582851240367, -13.142559126014115, 1.6054725485355685]
 ⋮
 [-3.1896192269450108, -11.548814571049508, -0.04644383549558256]
 [-3.329625018915606, -11.606369580558226, 0.05664949580691614]
 [-3.474740137276325, -11.656725930886232, 0.14271427273270487]
 [-3.6365284986310993, -11.69473692116042, 0.2541713712546988]
 [-3.752482744839304, -11.72737267252229, 0.3250555031674181]
 [-3.890169422548446, -11.74276337704782, 0.4146334759235643]
 [-4.045500932166366, -11.76306565085965, 0.49991647668516725]

# Convert data format

In [173]:
using DataFrames
using CSV
using LinearAlgebra

In [166]:
function jld_to_csv(filepath_jld)
    data = load(filepath_jld)

    df = DataFrame(time=Float64[], sun_x=Float64[], sun_y=Float64[], sun_z=Float64[], sun_dist=Float64[])

    for (t, r☉) in zip(data["et_range"], data["sun"])
        push!(df, [t, r☉..., norm(r☉)])
    end

    filepath_time_sun   = splitext(filepath_jld)[1] * "_time_sun.csv"
    filepath_surf_temps = splitext(filepath_jld)[1] * "_surf_temps.csv"
    CSV.write(filepath_time_sun, df)
    CSV.write(filepath_surf_temps, DataFrame(data["surf_temps"], :auto))

    df
end;

In [170]:
filepath_jld = "/Users/masanorikanamaru/Dropbox/Anivid/TPM_Ryugu_2018-07-01_2018-08-01/TPM_Ryugu.jld2"
jld_to_csv(filepath_jld);